In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

In [2]:
viirs = pd.read_csv("fire_nrt_V1_83401.csv")
# modis = pd.read_csv("fire_nrt_M6_83400.csv")

In [3]:
viirs.head()

latitude  longitude  bright_ti4  scan  track    acq_date  acq_time  \
0  33.03452  -83.08730       318.6  0.32   0.55  2019-07-01       636   
1  34.63658  -87.09289       304.2  0.56   0.69  2019-07-01       636   
2  34.63654  -87.08815       304.2  0.56   0.69  2019-07-01       636   
3  34.34694  -82.77595       306.3  0.32   0.54  2019-07-01       636   
4  43.24686  -70.96125       301.8  0.39   0.36  2019-07-01       636   

  satellite instrument confidence version  bright_ti5  frp daynight  
0         N      VIIRS          n  1.0NRT       288.2  0.9        N  
1         N      VIIRS          n  1.0NRT       286.8  1.0        N  
2         N      VIIRS          n  1.0NRT       287.2  1.9        N  
3         N      VIIRS          n  1.0NRT       291.0  0.8        N  
4         N      VIIRS          n  1.0NRT       285.4  0.6        N

In [4]:
# modis.head()

In [5]:
# modis = modis.rename(columns = {"bright_t31": "channel_brightness"})
# viirs = viirs.rename(columns = {"bright_ti5": "channel_brightness"})

In [6]:
# viirs = viirs.rename(columns = {"bright_ti4": "brightness"})

In [7]:
# firedata = pd.concat([viirs, modis])

In [8]:
# firedata = firedata.reset_index()

In [9]:
# firedata = firedata.drop(columns = "index")

In [10]:
# type(firedata['acq_date'][1])

In [11]:
viirs['acq_date'] = viirs['acq_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [12]:
viirs.head()

latitude  longitude  bright_ti4  scan  track   acq_date  acq_time  \
0  33.03452  -83.08730       318.6  0.32   0.55 2019-07-01       636   
1  34.63658  -87.09289       304.2  0.56   0.69 2019-07-01       636   
2  34.63654  -87.08815       304.2  0.56   0.69 2019-07-01       636   
3  34.34694  -82.77595       306.3  0.32   0.54 2019-07-01       636   
4  43.24686  -70.96125       301.8  0.39   0.36 2019-07-01       636   

  satellite instrument confidence version  bright_ti5  frp daynight  
0         N      VIIRS          n  1.0NRT       288.2  0.9        N  
1         N      VIIRS          n  1.0NRT       286.8  1.0        N  
2         N      VIIRS          n  1.0NRT       287.2  1.9        N  
3         N      VIIRS          n  1.0NRT       291.0  0.8        N  
4         N      VIIRS          n  1.0NRT       285.4  0.6        N

In [13]:
# Viirs in North America
viirs_na = viirs[(viirs.latitude <= 50) &
                 (viirs.latitude >= 28) &
                 (viirs.longitude <= -60) &
                 (viirs.longitude >= -130)]
viirs_na.head()
len(viirs_na)

143937

In [14]:
len(viirs)

239299

In [15]:
len(viirs.groupby(['latitude']))

208360

In [16]:
len(viirs.groupby(['longitude']))

217639

In [17]:
# Created smaller sample to test with
viirs_test = viirs[(viirs.latitude <= 50) &
                 (viirs.latitude >= 48) &
                 (viirs.longitude <= -120) &
                 (viirs.longitude >= -121)]
len(viirs_test)

122

In [18]:
viirs_test.head()
viirs_test
viirs_test = viirs_test.reset_index()
viirs_test = viirs_test.drop(columns="index")

In [19]:
viirs_test['acq_date'].value_counts(sort=False)

2019-09-21     1
2019-08-21     5
2019-08-09     7
2019-10-29     1
2019-08-28     4
2019-09-04     2
2019-08-06     6
2019-08-01     4
2019-08-08    22
2019-08-27     3
2019-07-27     6
2019-09-03     3
2019-10-23     1
2019-08-29    14
2019-09-05     1
2019-07-24     1
2019-07-31     7
2019-09-07     3
2019-08-07    20
2019-09-26     3
2019-09-02     3
2019-08-02     5
Name: acq_date, dtype: int64

In [23]:
# Test
# Determine chunks of consecutive days in dataset
dt = viirs_test['acq_date']
day = pd.Timedelta('1d')
# TODO need to check for multiple rows shift
in_block = ((dt - dt.shift(-1)).abs() == day) | (dt.diff() == day)
in_block

filt = viirs_test.loc[in_block]
breaks = filt['acq_date'].diff() != day
groups = breaks.cumsum()
groups

# for _, frame in filt.groupby(groups):
#     print(frame, end='\n\n')
    
viirs_test['groups'] = groups
viirs_test

latitude  longitude  bright_ti4  scan  track   acq_date  acq_time  \
0    48.04707 -120.46503       296.9  0.43   0.62 2019-07-24      1106   
1    48.26446 -120.72204       295.8  0.39   0.36 2019-07-27      1012   
2    48.26529 -120.72690       324.0  0.39   0.36 2019-07-27      1012   
3    48.26204 -120.72818       326.8  0.39   0.36 2019-07-27      1012   
4    48.26120 -120.72330       314.4  0.39   0.36 2019-07-27      1012   
5    48.26409 -120.73034       346.3  0.34   0.56 2019-07-27      2136   
6    48.26329 -120.72844       336.1  0.34   0.56 2019-07-27      2136   
7    48.26039 -120.72485       295.5  0.39   0.44 2019-07-31      1036   
8    48.53847 -120.22054       367.0  0.69   0.74 2019-07-31      2200   
9    48.53862 -120.23101       350.4  0.69   0.74 2019-07-31      2200   
10   48.53596 -120.23206       343.5  0.69   0.74 2019-07-31      2200   
11   48.54295 -120.23227       339.0  0.69   0.74 2019-07-31      2200   
12   48.53574 -120.22076       367.0  0.69   0.74 2019-07-31      2200   
13   48.54277 -120.22243       367.0  0.69   0.74 2019-07-31      2200   
14   48.25742 -120.72633       297.3  0.41   0.37 2019-08-01      1018   
15   48.26073 -120.72491       305.7  0.41   0.37 2019-08-01      1018   
16   48.26174 -120.72616       344.1  0.40   0.60 2019-08-01      2142   
17   48.26184 -120.72263       367.0  0.40   0.60 2019-08-01      2142   
18   48.26644 -120.73222       306.7  0.40   0.37 2019-08-02       954   
19   48.26564 -120.72642       322.3  0.40   0.37 2019-08-02       954   
20   48.26310 -120.73317       311.1  0.40   0.37 2019-08-02       954   
21   48.25897 -120.72832       315.0  0.40   0.37 2019-08-02       954   
22   48.25819 -120.72265       303.5  0.40   0.37 2019-08-02       954   
23   48.26060 -120.72910       296.8  0.45   0.39 2019-08-06      1024   
24   48.26500 -120.73235       305.1  0.45   0.39 2019-08-06      1024   
25   48.26746 -120.72578       300.6  0.45   0.39 2019-08-06      1024   
26   48.26989 -120.72466       341.5  0.47   0.64 2019-08-06      2148   
27   48.26983 -120.72842       356.4  0.47   0.64 2019-08-06      2148   
28   48.26381 -120.73018       345.5  0.47   0.64 2019-08-06      2148   
29   48.27203 -120.72083       323.8  0.38   0.36 2019-08-07      1006   
..        ...        ...         ...   ...    ...        ...       ...   
92   48.26063 -120.73402       307.7  0.43   0.38 2019-08-29       948   
93   48.25631 -120.72803       304.8  0.43   0.38 2019-08-29       948   
94   48.25965 -120.72903       297.9  0.73   0.76 2019-08-29      1130   
95   48.26130 -120.73332       302.5  0.73   0.76 2019-08-29      1130   
96   48.25608 -120.73497       321.3  0.73   0.76 2019-08-29      1130   
97   48.25507 -120.73851       303.7  0.73   0.76 2019-08-29      1130   
98   48.25336 -120.73407       329.1  0.73   0.76 2019-08-29      1130   
99   48.25774 -120.73930       296.7  0.73   0.76 2019-08-29      1130   
100  48.25437 -120.73048       315.9  0.73   0.76 2019-08-29      1130   
101  48.25722 -120.73542       351.4  0.41   0.45 2019-08-29      2118   
102  48.25758 -120.73109       328.2  0.41   0.45 2019-08-29      2118   
103  48.25562 -120.73379       356.8  0.41   0.45 2019-08-29      2118   
104  48.26133 -120.73635       313.3  0.41   0.37 2019-09-02      1018   
105  48.26463 -120.73488       297.4  0.41   0.37 2019-09-02      1018   
106  48.26556 -120.73998       300.3  0.41   0.37 2019-09-02      1018   
107  48.26286 -120.73766       325.3  0.40   0.37 2019-09-03       954   
108  48.25616 -120.73952       305.2  0.40   0.37 2019-09-03       954   
109  48.25952 -120.73862       322.6  0.40   0.37 2019-09-03       954   
110  48.25936 -120.74005       295.9  0.55   0.68 2019-09-04      1118   
111  48.26301 -120.73837       296.9  0.55   0.68 2019-09-04      1118   
112  48.26458 -120.73978       302.3  0.35   0.57 2019-09-05      1100   
113  48.26135 -120.73598       297.2  0.45   0.39 2019-09-07      1024   
114  48.2

In [26]:
# This function gets the current and next values for consecutive dates
# Returns a dataframe of Latitude and longitude values for consecutive days
def get_curr_values_and_next_values(df_in):
    result = []

    # Created dataframe within test range
    data_viirs_test = []
    consecutive_days = []
    for i in range(0, len(viirs_test)):
        # start at specific latitude and longitude, for first date
        lat_i = df_in.at[(i, 'latitude')]
        long_i = df_in.at[(i, 'longitude')]
        date_i = df_in.at[(i, 'acq_date')]
#         data_viirs_test.append([lat_i, long_i, date_i])

        if (i != 0): #base case
            if date_i == df_in.at[(i - 1, 'acq_date')]:
                continue;

        # Current day df
        df_curr_day = df[df_in.acq_date == date_i]
        df_curr_day = df_curr_day.reset_index()
        df_curr_day = df_curr_day.drop(columns = "index")

        # Get the next day
        # Work with days only
        next_date = date_i + timedelta(days=1)
        df_next_day = df[df_in.acq_date == next_date]
    #     print(date_i)

        # Reset index
        df_next_day = df_next_day.reset_index()
        df_next_day = df_next_day.drop(columns = "index")

        if (len(df_next_day) == 0):
            # No values for the next day
            # Fire has to last at least a day
            continue;

        else:
            consecutive_days.append([date_i]) # add current date
            consec_days_df = pd.DataFrame(consecutive_days, columns=['date'])
            
            result.append([df_curr_day, date_i])
            result.append([df_next_day, next_date])
        
        result_df = pd.DataFrame(result, columns = ['day_df', 'date'])
        
    return result_df


In [27]:
consec_values = get_curr_values_and_next_values(viirs_test)
consec_values

# Drop duplicate days
consec_day_values = consec_values.drop_duplicates(subset = "date")
# Latitude and longitude values for consecutive days ==> use this to get size
consec_day_values = consec_day_values.reset_index()
consec_day_values = consec_day_values.drop(columns = "index")
consec_day_values

NameError: name 'df' is not defined

In [384]:
consec_day_values.at[(9, "day_df")]

latitude  longitude       date
0   48.27109 -120.73183 2019-08-29
1   48.25713 -120.73469 2019-08-29
2   48.26063 -120.73402 2019-08-29
3   48.25631 -120.72803 2019-08-29
4   48.25965 -120.72903 2019-08-29
5   48.26130 -120.73332 2019-08-29
6   48.25608 -120.73497 2019-08-29
7   48.25507 -120.73851 2019-08-29
8   48.25336 -120.73407 2019-08-29
9   48.25774 -120.73930 2019-08-29
10  48.25437 -120.73048 2019-08-29
11  48.25722 -120.73542 2019-08-29
12  48.25758 -120.73109 2019-08-29
13  48.25562 -120.73379 2019-08-29

In [472]:
def size_of_fire(df_in, day):
    size_list = [] #general location, date, size
    
    # size of range to check
    lat_long_range = 0.001
    
    day_test = df_in.at[(day, "day_df")]
    date = df_in.at[(day, "date")]
    print(day_test)

    for j in range(0, len(day_test)):
        lat_j = day_test.at[(j, 'latitude')]
        long_j = day_test.at[(j, 'longitude')]
        radius = 1
        fire_locations = []
        for k in range(j + 1, len(day_test)):
            lat_k = day_test.at[(k, 'latitude')]
            long_k = day_test.at[(k, 'longitude')]
            
            if abs(lat_j - lat_k) >= lat_long_range and abs(long_j - long_k) >= lat_long_range: #base case
                # if out of range = different fires; do nothing
                x = 0
            else:
                fire_locations.append([lat_k, long_k]) #which points in fire
                radius += 1
                print(fire_locations)
        fire_locations_df = pd.DataFrame(fire_locations, columns = ['lat', 'long'])
        size_list.append([lat_j, long_j, radius, fire_locations_df, date])
            
    size_df = pd.DataFrame(size_list, columns = ['lat', 'long', 'size', 'fire_locations_df', 'date'])           
    return size_df

size_of_fire(consec_day_values, 1)

   latitude  longitude       date
0  48.25742 -120.72633 2019-08-01
1  48.26073 -120.72491 2019-08-01
2  48.26174 -120.72616 2019-08-01
3  48.26184 -120.72263 2019-08-01
[[48.26174, -120.72616000000001]]
[[48.26184, -120.72263000000001]]


lat       long  size                               fire_locations_df  \
0  48.25742 -120.72633     2           lat       long
0  48.26174 -120.72616   
1  48.26073 -120.72491     1  Empty DataFrame
Columns: [lat, long]
Index: []   
2  48.26174 -120.72616     2           lat       long
0  48.26184 -120.72263   
3  48.26184 -120.72263     1  Empty DataFrame
Columns: [lat, long]
Index: []   

        date  
0 2019-08-01  
1 2019-08-01  
2 2019-08-01  
3 2019-08-01

In [485]:
def get_size(consec_day_values, day):
    size_of_fire_day = size_of_fire(consec_day_values, day)
    for i in range(0, len(size_of_fire1)):
        lat_i = size_of_fire1.at[(i, 'lat')]
        long_i = size_of_fire1.at[(i, 'long')]
        df_i = size_of_fire1.at[(i, 'fire_locations_df')]
        print(df_i)
        if (df_i.empty == True):
            continue;
        else:
            print("do something to add size") #combine

for i in range(0, len(consec_day_values)):
    get_size(consec_day_values, i)

   latitude  longitude       date
0  48.26039 -120.72485 2019-07-31
1  48.53847 -120.22054 2019-07-31
2  48.53862 -120.23101 2019-07-31
3  48.53596 -120.23206 2019-07-31
4  48.54295 -120.23227 2019-07-31
5  48.53574 -120.22076 2019-07-31
6  48.54277 -120.22243 2019-07-31
[[48.53862, -120.23101000000001]]
[[48.53862, -120.23101000000001], [48.535740000000004, -120.22076000000001]]
[[48.54295, -120.23227]]
[[48.54295, -120.23227], [48.535740000000004, -120.22076000000001]]
[[48.542770000000004, -120.22243]]
        lat       long
0  48.26174 -120.72616
do something to add size
Empty DataFrame
Columns: [lat, long]
Index: []
        lat       long
0  48.26184 -120.72263
do something to add size
Empty DataFrame
Columns: [lat, long]
Index: []
   latitude  longitude       date
0  48.25742 -120.72633 2019-08-01
1  48.26073 -120.72491 2019-08-01
2  48.26174 -120.72616 2019-08-01
3  48.26184 -120.72263 2019-08-01
[[48.26174, -120.72616000000001]]
[[48.26184, -120.72263000000001]]
        lat    

        lat       long
0  48.26174 -120.72616
do something to add size
Empty DataFrame
Columns: [lat, long]
Index: []
        lat       long
0  48.26184 -120.72263
do something to add size
Empty DataFrame
Columns: [lat, long]
Index: []
   latitude  longitude       date
0  48.25885 -120.73291 2019-08-27
1  48.25939 -120.73291 2019-08-27
2  48.26410 -120.73628 2019-08-27
[[48.25939, -120.73291]]
        lat       long
0  48.26174 -120.72616
do something to add size
Empty DataFrame
Columns: [lat, long]
Index: []
        lat       long
0  48.26184 -120.72263
do something to add size
Empty DataFrame
Columns: [lat, long]
Index: []
   latitude  longitude       date
0  48.26533 -120.73811 2019-08-28
1  48.26125 -120.73445 2019-08-28
2  48.25800 -120.73575 2019-08-28
3  48.26137 -120.73659 2019-08-28
[[48.26137, -120.73658999999999]]
[[48.26137, -120.73658999999999]]
        lat       long
0  48.26174 -120.72616
do something to add size
Empty DataFrame
Columns: [lat, long]
Index: []
        lat